In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers_interpret import SequenceClassificationExplainer

# Instanciando modelo

In [3]:
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
CHECKPOINT_PATH = "../../data/VICTOR/checkpoints/checkpoint-149136"

In [4]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(CHECKPOINT_PATH)

# Carregando exemplos para predição

In [6]:
import pandas as pd

In [23]:
TRAIN_PATH = "/home/kenzosaki/mestrado/data/VICTOR/train_small.csv"
TEST_PATH = "/home/kenzosaki/mestrado/data/VICTOR/test_small.csv"

In [24]:
train_df = pd.read_csv(TRAIN_PATH, index_col=0)
test_df = pd.read_csv(TEST_PATH, index_col=0)

In [36]:
label_encoder = LabelEncoder()
label_encoder.fit(train_df["document_type"])

LabelEncoder()

In [37]:
label_encoder.classes_

array(['acordao_de_2_instancia', 'agravo_em_recurso_extraordinario',
       'despacho_de_admissibilidade', 'outros', 'peticao_do_RE',
       'sentenca'], dtype=object)

# Realizando inferências

In [38]:
from sklearn.preprocessing import LabelEncoder

In [75]:
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer,
    custom_labels=label_encoder.classes_
)

In [80]:
FILTER_BY = "acordao_de_2_instancia"

In [87]:
sampled_df = test_df[test_df["document_type"] == FILTER_BY]
example = sampled_df.sample(1)
ex_text = example["body"].values[0]
ex_label = example["document_type"].values[0]
ex_text, ex_label

('vistos etc decide segunda turma recursal dos juizados especiais federais pernambuco unanimidade negar provimento recurso nos termos voto acima recife data julgamento jorge andré carvalho mendonça juiz federal relator',
 'acordao_de_2_instancia')

In [88]:
word_attributions = cls_explainer(ex_text, class_name=ex_label)

In [89]:
_ = cls_explainer.visualize()